In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)
cwd = '/content/drive/MyDrive/BookingcomChallenge2021/'

import os
os.chdir(cwd)

Mounted at /content/drive


In [67]:
import argparse
import torch
import legdata as data
from tqdm import tqdm

In [68]:
parser = argparse.ArgumentParser(description='PyTorch Wikitext-2 Language Model')

# Model parameters.
parser.add_argument('--data', type=str, default='./data/wikitext-2',
                    help='location of the data corpus')
parser.add_argument('--checkpoint', type=str, default='./model.pt',
                    help='model checkpoint to use')
parser.add_argument('--outf', type=str, default='generated.txt',
                    help='output file for generated text')
parser.add_argument('--words', type=int, default='1000',
                    help='number of words to generate')
parser.add_argument('--bptt', type=int, default=35,
                    help='sequence length')
parser.add_argument('--seed', type=int, default=None,
                    help='random seed')
parser.add_argument('--cuda', action='store_true',
                    help='use CUDA')
parser.add_argument('--temperature', type=float, default=1.0,
                    help='temperature - higher will increase diversity')
parser.add_argument('--log-interval', type=int, default=100,
                    help='reporting interval')

_StoreAction(option_strings=['--log-interval'], dest='log_interval', nargs=None, const=None, default=100, type=<class 'int'>, choices=None, help='reporting interval', metavar=None)

In [97]:
args = parser.parse_args(args=["--data", "../booking_com_data/legs_4legs",\
                               "--checkpoint", "../models/lstm_transformer.pth",\
                               "--outf", "../result/generated_by_lstm_transformer.txt",\
                               "--words", "1",\
                               "--bptt", "4",\
                               "--temperature", "1.0",\
                               "--cuda"])

In [98]:
# Set the random seed manually for reproducibility.
if args.seed:
    torch.manual_seed(args.seed)

if torch.cuda.is_available():
    if not args.cuda:
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")

device = torch.device("cuda" if args.cuda else "cpu")
device

device(type='cpu')

In [99]:
if args.temperature < 1e-3:
    parser.error("--temperature has to be greater or equal 1e-3")

with open(args.checkpoint, 'rb') as f:
    model = torch.load(f, map_location='cpu').to(device)
model.eval()

RNNModel(
  (drop): Dropout(p=0.1, inplace=False)
  (encoder): Embedding(39878, 512)
  (rnn): GRU(512, 1024, num_layers=3, dropout=0.1)
  (decoder): Linear(in_features=1024, out_features=39878, bias=True)
)

In [100]:
corpus = data.Corpus(args.data)
ntokens = len(corpus.dictionary)

model_type = model.model_type if hasattr(model, 'model_type') else None

In [101]:
test_data = corpus.test.view(-1, 4)

In [104]:
n_correct = 0

if model_type == 'LSTMTransformer':
    hidden = model.init_hidden(1)
    mems = None
elif model_type == 'Transformer':
    pass
else:
    hidden = model.init_hidden(1)

with open(args.outf, 'w') as outf:
    with torch.no_grad():  # no tracking history
        for i, q in enumerate(tqdm(test_data)):
            input = q[:-1].view(-1, 1)
            if model_type == "LSTMTransformer":
                output, _, _ = model(input, hidden, mems)
            elif model_type == 'Transformer':
                output = model(input, False)
            else:
                output, _ = model(input, hidden)
            word_weights = output[-1].squeeze().div(args.temperature).exp().cpu()
            word_indices = torch.argsort(word_weights, descending=True)[:4]
            if q[-1] in word_indices:
                n_correct += 1

IndentationError: expected an indented block (<ipython-input-104-f152cec4eb27>, line 20)

In [96]:
n_correct/len(test_data)

0.03783731219915398